In [41]:
#Find protein homologs by computing levenshtein distance via dynamic programming
import numpy as np

aa=['G','A','V','L','I','S','C','T','M','P','F','Y','W','H','K','R','D','E','N','Q','X']

classes=['aliphatic','aliphatic','aliphatic','aliphatic','aliphatic','Hydroxyl','Hydroxyl','Hydroxyl','Hydroxyl','Cyclic','Aromatic','Aromatic','Aromatic','Basic','Basic','Basic','Acidic','Acidic','Acidic','Acidic','Any']
aa_dict = {}
for i in range(len(aa)):
    aa_dict[aa[i]] = classes[i]


def levenshtein(sequence1,sequence2):
    
    if len(sequence1)<len(sequence2):
        return levenshtein(sequence2,sequence1)
    
    if len(sequence2)==0:
        return len(sequence1)
    memo=np.zeros((len(sequence1)+1,len(sequence2)+1),np.int8)
    
    memo[0]=range(len(sequence2)+1)
    
    for i,c1 in enumerate(sequence1):
        memo[i+1, 0]=i+1
        for j,c2 in enumerate(sequence2):
            insertions=memo[i,j+1]+1
            deletions=memo[i+1,j]+1
            if aa_dict[sequence1[i]] != aa_dict[sequence2[j]]:
                substitutions=memo[i,j]+2*(c1 != c2)
            elif aa_dict[sequence1[i]] == aa_dict[sequence2[j]]:
                substitutions=memo[i,j]+(c1 != c2)
            memo[i+1,j+1]=min(insertions,deletions,substitutions)
    return memo[-1,-1]



In [12]:
#Read in fasta file
seq_dict={}
with open('proteins.fasta', "r") as file:
    for line in file:
        if line.startswith('>'):
            x=line.strip('\n')
            y=next(file)
            y=y.strip('\n')
            seq_dict[x] = y

#test levenshtein
seq1=seq_dict['>NP_000940.1 prolactin receptor isoform 1 precursor [Homo sapiens]']
seq2=['P','I','D','N','Y','L','K','L','L','K','C','R','I','I','H','N','N','N','C']

levenshtein(seq1,seq2)

66

In [42]:
import pandas as pd
def output_k_homologs(reference_sequence,k):
    new_seq_dict=dict((x, v) for x, v in seq_dict.items() if v != reference_sequence)
    results=pd.DataFrame([])
    for i,j in new_seq_dict.items():
        results = results.append(pd.DataFrame({'proteinName': i, 'levenshteinDistance': levenshtein(reference_sequence,j)}, index=[0]), ignore_index=True)
    return results.sort_values(by='levenshteinDistance').head(k)

        
        

In [43]:
#output homologs
output_k_homologs(seq1, 10)


,levenshteinDistance,proteinName
768,11,>NP_001254666.1 prolactin receptor precursor [...
728,17,>NP_001001868.1 prolactin receptor precursor [...
753,20,>NP_001075700.1 prolactin receptor precursor [...
737,27,>NP_001272598.1 prolactin receptor precursor [...
723,27,>NP_001034815.1 prolactin receptor long form p...
725,27,>NP_776580.1 prolactin receptor short form pre...
731,33,>NP_036762.1 prolactin receptor isoform b prec...
729,33,>NP_001240711.1 prolactin receptor isoform 3 p...
727,33,>NP_001029283.1 prolactin receptor isoform a p...
724,33,>NP_035299.4 prolactin receptor isoform 1 prec...
